In [1]:
from pydantic import BaseModel, Field
# Import things that are needed generically
from langchain.chains import LLMMathChain
from langchain.utilities import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [2]:
import os
OPEN_AI_KEY = os.getenv("OPEN_AI_KEY")
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")

In [5]:
llm = ChatOpenAI(temperature=0)

In [6]:
tools = []

class Addition:
    def __init__():
        pass
    def run(x, y):
        return x + y
    class SchemaInput(BaseModel):
        x: int = Field() 
        y: int = Field()

from pydantic import BaseModel, Field

tools.append(
    Tool.from_function(
        func=Addition.run,
        name="Addition",
        description="useful for when you need to answer questions about math",
        args_schema=Addition.SchemaInput
        # coroutine= ... <- you can specify an async method if desired as well
    )
)

# tools = [
#     Tool.from_function(
#         func=search.run,
#         name="Search",
#         description="useful for when you need to answer questions about current events"
#         # coroutine= ... <- you can specify an async method if desired as well
#     ),
# ]


ValidationError: 1 validation error for Tool
args_schema
  subclass of BaseModel expected (type=type_error.subclass; expected_class=BaseModel)

In [ ]:
# pip install langchain
# pip install google-search-results
# pip install openai

from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain.utilities import SerpAPIWrapper
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
import re
import os
from getpass import getpass

from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.agents.agent_toolkits import FileManagementToolkit

# from tempfile import TemporaryDirectory

# # We'll make a temporary directory to avoid clutter
# working_directory = TemporaryDirectory()
from collections import namedtuple
workdir = namedtuple('working_directory', ('name'))
working_directory= workdir(name = './')                        

# toolkit = FileManagementToolkit(
#     root_dir=str(working_directory.name)
# )  # If you don't provide a root_dir, operations will default to the current working directory
# toolkit.get_tools()



tools = FileManagementToolkit(
    root_dir=str(working_directory.name),
    selected_tools=["read_file", "write_file", "list_directory"],
).get_tools()
# tools

OPEN_AI_KEY = os.getenv("OPEN_AI_KEY")
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")
# SERPAPI_API_KEY = getpass()

# Define which tools the agent can use to answer user queries
# search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
# tools = [
#     Tool(
#         name="Search",
#         func=search.run,
#         description="useful for when you need to answer questions about current events"
#     ),

# ]




# Set up the base template
template = """Complete the objective as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

These were previous tasks you completed:



Begin!

Question: {input}
{agent_scratchpad}"""
# template = """Complete the objective as best you can. You have access to the following tools:

# {tools}

# Use the following format:

# Question: the input question you must answer
# Thought: you should always think about what to do
# Action: the action to take, should be one of [{tool_names}]
# Action Input: the input to the action
# Observation: the result of the action
# ... (this Thought/Action/Action Input/Observation can repeat N times)
# Thought: I now know the final answer
# Final Answer: the final answer to the original input question

# These were previous tasks you completed:



# Begin!

# Question: {input}
# {agent_scratchpad}"""

# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()

# OPENAI_API_KEY = getpass()

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0)

# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
agent_executor.run("Search for Leo DiCaprio's girlfriend on the internet.")

In [ ]:
## This notebook is designed to provide a chat interface that will allow the user to 'watch' as the agent looks 
## at the directory, asks to read them (or is given explicit permission), and then reads them, one by one. 
## it reads it one by one, it offers suggestions per improvement. 
## It can 'list' the files in the directory.
## It can 'read' the files in the directory.
## It can 'suggest' changes to the files in the directory.
## It can 'write' the changes to the files in the directory.
## It can 'commit' the changes to the files in the directory.
## It can 'help' the user understand what it can do.
## It periodically reminds to commit the changes.

import os

def run_loop():
# It uses langchain. 
    response = 'Start'
        while response.lower() is not 'stop': 
            response = input('What would you like to do?')
            
            if response == 'list':
                print('Listing files in directory...')
                # list files in directory
                return_text = prompt_for(list_files, response)
            elif response == 'read_file':
                print('Reading files in directory...')
                # read files in directory
                return_text = prompt_for(, response)



In [ ]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from networkx.algorithms import community
import networkx

In [ ]:
degrees = dict(networkx.degree(G))


In [8]:
from typing import Any, Dict

from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI
from langchain.tools.requests.tool import RequestsGetTool, TextRequestsWrapper
from pydantic import Field, root_validator
from pydantic.v1 import BaseModel
llm = OpenAI(temperature=0)

# !pip install tldextract > /dev/null

    
    # [notice] A new release of pip is available: 23.0.1 -> 23.1
    # [notice] To update, run: pip install --upgrade pip

import tldextract

_APPROVED_DOMAINS = {
    "langchain",
    "wikipedia",
}


class ToolInputSchema(BaseModel):
    url: str = Field(...)

    @root_validator(skip_on_failure=True)
    def validate_query(cls, values: Dict[str, Any]) -> Dict:
        url = values["url"]
        domain = tldextract.extract(url).domain
        if domain not in _APPROVED_DOMAINS:
            raise ValueError(
                f"Domain {domain} is not on the approved list:"
                f" {sorted(_APPROVED_DOMAINS)}"
            )
        return values


tool = RequestsGetTool(
    args_schema=ToolInputSchema, requests_wrapper=TextRequestsWrapper()
)

agent = initialize_agent(
    [tool], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)

# This will succeed, since there aren't any arguments that will be triggered during validation
answer = agent.run("What's the main title on langchain.com?")
print(answer)

    # The main title of langchain.com is "LANG CHAIN 🦜️🔗 Official Home Page"

agent.run("What's the main title on google.com?")

TypeError: Object of type 'FieldInfo' is not JSON serializable

In [ ]:
TypeError: Object of type 'FieldInfo' is not JSON serializable
